## Badanie wpływu szumu na proces uczenia 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import pandas as pd
from sklearn import svm
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

In [2]:
def fun(x,y):
    return np.log10(np.abs(x))*5*np.cos(y)+(0.55*(x+y))

In [3]:
x_n = np.outer(np.linspace(-10, 10, 100), np.ones(100))
y_n = x_n.copy().T
z_n = fun(x_n,y_n)

In [4]:
models = [
    { 'kernel': 'linear', 'C': 64.76855, 'epsilon': 0.006793}, ##linear_mae
    { 'kernel': 'linear', 'C': 21.62282, 'epsilon': 0.01343 }, ##linear_mse
    { 'kernel': 'linear', 'C': 6.09208, 'epsilon': 0.00285},##linear_r2
    { 'kernel': 'rbf', 'C': 7.89739, 'epsilon': 0.020708, 'gamma': 30.10516},#rbf_mae
    { 'kernel': 'rbf', 'C': 7.714452, 'epsilon': 0.01036, 'gamma': 24.57617 }, ##rbf_mse, 
    {  'kernel': 'rbf', 'C': 62.90105, 'epsilon': 0.00019, 'gamma': 21.73123}, ##rbf_r2
    { 'kernel': 'poly', 'C': 98.05169, 'epsilon': 0.01403, 'degree': 6, 'coef0': 9.89688},##poly_mae,
    { 'kernel': 'poly', 'C': 52.47482, 'epsilon': 0.00283, 'degree': 7, 'coef0': 9.45739}, ##poly_mse
    { 'kernel': 'poly', 'C': 0.34617, 'epsilon': 0.411307, 'degree': 4, 'coef0': 2.2253}, ##poly_r2
    { 'kernel': 'sigmoid', 'C': 70.96374, 'epsilon': 1.4122, 'coef0': 0.7384},##sigmoid_mae
    { 'kernel': 'sigmoid', 'C': 89.26535, 'epsilon': 0.05399, 'coef0': 7.50992}, ##sigmoid_mse
    {'kernel': 'sigmoid', 'C': 50.36861, 'epsilon': 1.50263, 'coef0': 0.87763} ##sigmoid_r2
]

In [4]:
noise_levels = [0.01, 0.05, 0.1, 0.5, 1.0, 5.0]
results = []

In [6]:
for level in noise_levels:
    results = []
    for params in models: 
  
        noise = np.random.normal(0,level,(100,100))
        noised_z = z_n + noise
        

        Xn = np.column_stack((x_n.flatten(), y_n.flatten()))
        yn = noised_z.flatten()

        Xn_train, Xn_test, yn_train, yn_test = train_test_split(Xn, yn, test_size=0.1, random_state=85)

        Xn_scaler = StandardScaler()
        Xn_train = Xn_scaler.fit_transform(Xn_train)
        Xn_test = Xn_scaler.transform(Xn_test)

        yn_scaler = StandardScaler()
        yn_train = yn_scaler.fit_transform(yn_train[:,None])[:,0]
        yn_test = yn_scaler.transform(yn_test[:,None])[:,0]
    
        model = svm.SVR(**params, verbose=2)
        model.fit(Xn_train, yn_train)

        yn_pred_train = model.predict(Xn_train)
        yn_pred_test = model.predict(Xn_test)
        
        r2_train = model.score(Xn_train, yn_train)
        r2_test = model.score(Xn_test, yn_test)
        print(f"R2 na zbiorze trenującym: {r2_train}")
        print(f"R2 na zbiorze testującym: {r2_test}")

        mse_train = metrics.mean_squared_error(yn_train, yn_pred_train)
        mse_test = metrics.mean_squared_error(yn_test, yn_pred_test)

        print(f"MSE na zbiorze trenującym: {mse_train}")
        print(f"MSE na zbiorze testującym: {mse_test}")

        mae_train = metrics.mean_absolute_error(yn_train, yn_pred_train)
        mae_test = metrics.mean_absolute_error(yn_test, yn_pred_test)

        print(f"MAE na zbiorze trenującym: {mae_train}")
        print(f"MAE na zbiorze testującym: {mae_test}")
        results.append({'params': params, 'noise': level, 'r2_train': r2_train, 'r2_test': r2_test, 'mse_train': mse_train, 'mse_test': mse_test, 'mae_test': mae_test, 'mae_train': mae_train})
    name = './noises_results_'+str(level)+'.csv'
    df = pd.DataFrame(results)
    df.to_csv(name)

[LibSVM]R2 na zbiorze trenującym: 0.7540681943663333
R2 na zbiorze testującym: 0.7283998859228769
MSE na zbiorze trenującym: 0.24593180563366665
MSE na zbiorze testującym: 0.2504168538726807
MAE na zbiorze trenującym: 0.41349739380600864
MAE na zbiorze testującym: 0.4159088040243553
[LibSVM]R2 na zbiorze trenującym: 0.7539557977721278
R2 na zbiorze testującym: 0.7265328663948375
MSE na zbiorze trenującym: 0.2460442022278722
MSE na zbiorze testującym: 0.2554907908795335
MAE na zbiorze trenującym: 0.41479198704908726
MAE na zbiorze testującym: 0.41999788418617984
[LibSVM]R2 na zbiorze trenującym: 0.7530894780451474
R2 na zbiorze testującym: 0.7214219978889267
MSE na zbiorze trenującym: 0.24691052195485266
MSE na zbiorze testującym: 0.2567694696800641
MAE na zbiorze trenującym: 0.4149732065147714
MAE na zbiorze testującym: 0.4193334138848761
[LibSVM]R2 na zbiorze trenującym: 0.991419464906735
R2 na zbiorze testującym: 0.9884696949113506
MSE na zbiorze trenującym: 0.008580535093264997
MSE 

[LibSVM]R2 na zbiorze trenującym: 0.536117973669839
R2 na zbiorze testującym: 0.41649211256176455
MSE na zbiorze trenującym: 0.46388202633016096
MSE na zbiorze testującym: 0.5869543784252476
MAE na zbiorze trenującym: 0.5145608545169471
MAE na zbiorze testującym: 0.6051967143596301
[LibSVM]R2 na zbiorze trenującym: -738.0439424227906
R2 na zbiorze testującym: -768.5364470223145
MSE na zbiorze trenującym: 739.0439424227906
MSE na zbiorze testującym: 753.4054281606756
MAE na zbiorze trenującym: 20.288840909017843
MAE na zbiorze testującym: 20.678239083523604
[LibSVM]R2 na zbiorze trenującym: -26980.50298685413
R2 na zbiorze testującym: -25514.742087181516
MSE na zbiorze trenującym: 26981.50298685413
MSE na zbiorze testującym: 24002.66707989584
MAE na zbiorze trenującym: 106.76818842513987
MAE na zbiorze testującym: 100.8201997236055
[LibSVM]R2 na zbiorze trenującym: 0.416631287058975
R2 na zbiorze testującym: 0.3822057565440089
MSE na zbiorze trenującym: 0.583368712941025
MSE na zbiorze 

## Porównanie z innym modelem

In [5]:
from sklearn.linear_model import LinearRegression

In [8]:
results_ = []
for level in noise_levels:
    print(level)
    noise = np.random.normal(0,level,(100,100))
    noised_z = z_n + noise
        
    Xn = np.column_stack((x_n.flatten(), y_n.flatten()))
    yn = noised_z.flatten()
    Xn_train, Xn_test, yn_train, yn_test = train_test_split(Xn, yn, test_size=0.1, random_state=85)

    Xn_scaler = StandardScaler()
    Xn_train = Xn_scaler.fit_transform(Xn_train)
    Xn_test = Xn_scaler.transform(Xn_test)

    yn_scaler = StandardScaler()
    yn_train = yn_scaler.fit_transform(yn_train[:,None])[:,0]
    yn_test = yn_scaler.transform(yn_test[:,None])[:,0]
    
    model = LinearRegression(fit_intercept=False)
    model.fit(Xn_train, yn_train)

    yn_pred_train = model.predict(Xn_train)
    yn_pred_test = model.predict(Xn_test)
        
    r2_train = model.score(Xn_train, yn_train)
    r2_test = model.score(Xn_test, yn_test)
    print(f"R2 na zbiorze trenującym: {r2_train}")
    print(f"R2 na zbiorze testującym: {r2_test}")

    mse_train = metrics.mean_squared_error(yn_train, yn_pred_train)
    mse_test = metrics.mean_squared_error(yn_test, yn_pred_test)

    print(f"MSE na zbiorze trenującym: {mse_train}")
    print(f"MSE na zbiorze testującym: {mse_test}")

    mae_train = metrics.mean_absolute_error(yn_train, yn_pred_train)
    mae_test = metrics.mean_absolute_error(yn_test, yn_pred_test)

    print(f"MAE na zbiorze trenującym: {mae_train}")
    print(f"MAE na zbiorze testującym: {mae_test}")
    results.append({
        'noise': level, 
        'r2_train': r2_train, 
        'r2_test': r2_test, 
        'mse_train': mse_train, 
        'mse_test': mse_test, 
        'mae_test': mae_test, 
        'mae_train': mae_train
    })

name = './noises_results_linear.csv'
df = pd.DataFrame(results)
df.to_csv(name)

0.01
R2 na zbiorze trenującym: 0.7604533945355006
R2 na zbiorze testującym: 0.7327587426174093
MSE na zbiorze trenującym: 0.2395466054644994
MSE na zbiorze testującym: 0.24761713772585167
MAE na zbiorze trenującym: 0.40905182645918303
MAE na zbiorze testującym: 0.4131821187609505
0.05
R2 na zbiorze trenującym: 0.7604592169040867
R2 na zbiorze testującym: 0.7325041595346078
MSE na zbiorze trenującym: 0.23954078309591326
MSE na zbiorze testującym: 0.24788603662398165
MAE na zbiorze trenującym: 0.4090972108211814
MAE na zbiorze testującym: 0.41347293827486153
0.1
R2 na zbiorze trenującym: 0.7603998096080995
R2 na zbiorze testującym: 0.7323345907785865
MSE na zbiorze trenującym: 0.23960019039190053
MSE na zbiorze testującym: 0.24847432020566368
MAE na zbiorze trenującym: 0.4090389267726003
MAE na zbiorze testującym: 0.41380004907981127
0.5
R2 na zbiorze trenującym: 0.7549692372046651
R2 na zbiorze testującym: 0.7229741588723813
MSE na zbiorze trenującym: 0.24503076279533495
MSE na zbiorze 